## Freezing layers

In this notebook we will try to freeze layers and check if that improves performance

In [1]:
%load_ext autoreload
%autoreload 2
import os
from datetime import datetime
import fire
import torch
import pandas as pd
from torchtext import data
import torch.nn as nn
from transformers import (
    AdamW, BertForSequenceClassification, BertTokenizer,
    get_constant_schedule_with_warmup
)

from offenseval.nn import (
    Tokenizer,
    train, evaluate, train_cycle, save_model, load_model, evaluate_dataset
)
from offenseval.datasets import datasets

pd.options.display.max_rows = 200
pd.options.display.max_colwidth = 300

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Create fields and some other boilerplate

In [2]:

model, TEXT = load_model("../../models/bert_cased.all.pt", device)
model.eval();

Get the predictions

In [3]:
from offenseval.datasets import build_dataset
ID = data.LabelField(dtype = str, use_vocab=False, preprocessing=str)
SUBTASK_A = data.LabelField()

fields = {
    "text": ('text', TEXT),
}
test_path = "../../data/English/test.tsv"
test_dataset = build_dataset("../../data/English/test.tsv", fields)

In [5]:

print("Building iterators")

test_it = data.Iterator(
    test_dataset, batch_size=1, device=device,
    shuffle=False,
)


Building iterators


In [6]:
from tqdm.auto import tqdm

with torch.no_grad():
    predicted_labels = []

    for batch in tqdm(test_it):
        text, lens = batch.text
        
        predictions = model(text)
        predictions = torch.round(torch.sigmoid(predictions))

        predicted_labels.append(predictions)


NameError: name 'ids' is not defined

In [9]:

predicted_labels = torch.cat(predicted_labels).cpu().numpy()

In [25]:

df_test = pd.read_table(test_path, index_col=0)

In [26]:
df_test["pred"] = 'NOT'

df_test.loc[predicted_labels.reshape(-1) > 0, "pred"] = "OFF"

In [28]:
df_test.iloc[100:110]

,tweet,pred
id,,
A100,@USER you can be sure that now that I can talk to you on twitter I often love your photos,NOT
A101,MPs struggling to vote electronically is so surreal given the importance of what they're voting on! #Article370,NOT
A102,@USER They’re just sick in the head trolls who are more to be pitied! Chin up Nathan. 👍,OFF
A103,@USER Thank you for believing I'm good looking but I'm not actually.,NOT
A104,@USER @USER shut up and go back to your porn,OFF
A105,Already looking forward to the end of the day to when I have a coffee date with a friend after work.,NOT
A106,leaving us Not even a good luckLaurent Koscielny is leaving us to join Ligue 1 side Bordeaux,NOT
A107,@USER Add the #MiddleEastServers,NOT
A108,@USER @USER FYI I had to re-follow you and I think it's the 2nd time this has happened. What's up @USER,NOT
